In [1]:
import pandas as pd
import numpy as np
import random
import itertools
import copy

import seaborn as sns
import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import *

from sklearn.model_selection import train_test_split

from sklearn.datasets import make_classification

from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import (LinearRegression,LogisticRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor, Ridge)

from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

# import warnings
# from sklearn.exceptions import DataConversionWarning
# warnings.filterwarnings(action='ignore', category=DataConversionWarning)

import random
from sklearn.impute import KNNImputer

In [2]:
pd.options.display.max_columns = 120

In [3]:
dataset = pd.read_json('dataset')

# Skalujemy dane

In [4]:
columns, values = dataset.select_dtypes(include=['float64']).columns, dataset.select_dtypes(include=['float64']).values

min_max_scaler = MinMaxScaler()
values_scaled = min_max_scaler.fit_transform(values)

dataset = dataset.drop(columns = columns)

for i, column in enumerate(columns):
    dataset[column] = values_scaled.T[i]

In [5]:
dataset.head()

,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31,Średnie dobowe zachmurzenie ogólne [oktanty],Średnia dobowa prędkość wiatru [m/s],Średnia dobowe ciśnienie pary wodnej [hPa],Średnia dobowa wilgotność względna [%],Średnia dobowe ciśnienie na poziomie stacji [hPa],Suma opadu dzień [mm],Suma opadu noc [mm],Maksymalna temperatura dobowa [°C],Minimalna temperatura dobowa [°C],Suma dobowa opadu [mm],Temperatura minimalna przy gruncie [°C],Usłonecznienie [godziny],Średnia temperatura dobowa [°C],Czas trwania opadu deszczu [godziny],Czas trwania rosy [godziny],Czas trwania zamglenia [godziny]
0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.9250,0.3750,0.300429,0.829175,0.568069,0.800000,0.0,0.369835,0.478372,0.171429,0.495376,0.037037,0.437788,1.000000,0.0,0.621176
1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.9250,0.4750,0.326180,0.973129,0.613861,0.133333,0.2,0.380165,0.460560,0.057143,0.442536,0.080247,0.437788,0.844444,0.0,0.371765
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.9250,0.5625,0.317597,0.808061,0.582921,1.000000,1.0,0.425620,0.463104,1.000000,0.479524,0.000000,0.460829,1.000000,0.0,0.324706
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.9125,0.7625,0.356223,0.952015,0.626238,0.533333,1.0,0.448347,0.447837,0.342857,0.431968,0.000000,0.472350,0.859259,0.0,0.108235
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.5375,0.4125,0.300429,0.721689,0.600248,0.000000,0.6,0.421488,0.483461,0.085714,0.466314,0.283951,0.458525,0.000000,0.0,0.409412


# Wstępna analiza klasyfikatorów

# Model opadów ilościowy


In [6]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

In [7]:
estimators = [('OLS', LinearRegression(normalize=True, copy_X=True)),
              ('RANSAC', RANSACRegressor(random_state=42, max_trials=1000)),
              ('HuberRegressor', HuberRegressor(max_iter=10000)),
              ('Ridge', Ridge(normalize=True, copy_X=True))]

In [8]:
def linearEstimatorsPipe(datafame, features, estimators, days = 3,):
    """input:Cecha predykowana, dataset, dni"""
    """return:predykcja, wartość, mean_sq_error, accuracy"""
    predyctionByFeatures = {}
    meanSquareErrorByFeatures = {}
    accuracyByFeatures = {}
    
    for feature in features:
        predyctionByEstimators = {}
        meanSquareErrorByEstimators = {}
        accuracyByEstimators = {}
        
        y = datafame[feature]
        X = datafame.select_dtypes(include=['float64'])
#         X = datafame[feature]
        for name, estimator in estimators:
#             pipe = Pipeline(PolynomialFeatures(2), [(f'{name}', estimator)])
            pipe = make_pipeline(PolynomialFeatures(2), estimator)
            
            predictionByDays = {}
            meanSquareErrorByDays = {}
            accuracyByDays = {}
            
            for i in range(1,(days+1)):
                try:
                    X_train, X_test, y_train, y_test = train_test_split(X[:-i], y.shift(-i)[:-i],\
                                                                        test_size=0.2, random_state=12, stratify=y.shift(-i)[:-i])
                    pipe.fit(X_train, y_train)
                    mse = mean_squared_error(pipe.predict(X_test), y_test)
                    
                    predictionByDays[i] = pipe.predict(X_test)
                    predictionByDays["y_for"+str(i)] = y_test
                    meanSquareErrorByDays[i] = mean_squared_error(pipe.predict(X_test), y_test)
                    accuracyByDays[i] = pipe.score(X_test, y_test)
                except ValueError:
                    pass
            predyctionByEstimators[name] = predictionByDays
            meanSquareErrorByEstimators[name] = meanSquareErrorByDays
            accuracyByEstimators[name] = accuracyByDays
            print(f" Done {name}")
        predyctionByFeatures[feature] = predyctionByEstimators
        meanSquareErrorByFeatures[feature] = meanSquareErrorByEstimators
        accuracyByFeatures[feature] = accuracyByEstimators
        print(f" Done {feature}")
    return predyctionByFeatures,\
           meanSquareErrorByFeatures,\
           accuracyByFeatures

In [9]:
features_for_predict =  ('Średnie dobowe zachmurzenie ogólne [oktanty]', "Usłonecznienie [godziny]",\
                         'Średnia temperatura dobowa [°C]', 'Suma dobowa opadu [mm]')

In [10]:
predyction = linearEstimatorsPipe(dataset.select_dtypes(include=['float64']), features_for_predict, estimators)

 Done OLS
 Done RANSAC
 Done HuberRegressor
 Done Ridge
 Done Średnie dobowe zachmurzenie ogólne [oktanty]
 Done OLS
 Done RANSAC
 Done HuberRegressor
 Done Ridge
 Done Usłonecznienie [godziny]
 Done OLS
 Done RANSAC
 Done HuberRegressor
 Done Ridge
 Done Średnia temperatura dobowa [°C]
 Done OLS
 Done RANSAC
 Done HuberRegressor
 Done Ridge
 Done Suma dobowa opadu [mm]


In [11]:
predyction[2]["Średnia temperatura dobowa [°C]"]

{'OLS': {}, 'RANSAC': {}, 'HuberRegressor': {}, 'Ridge': {}}

In [12]:
predyction[2]["Usłonecznienie [godziny]"]

{'OLS': {1: 0.6517279968153971,
  2: 0.47707107564244766,
  3: 0.40529128747004295},
 'RANSAC': {1: -3.499726536023714,
  2: -7.477529943122586,
  3: -8.985191417514715},
 'HuberRegressor': {1: 0.6474838937116111,
  2: 0.45838606474793886,
  3: 0.3857120946726884},
 'Ridge': {1: 0.6225154132892747,
  2: 0.4498124953546345,
  3: 0.3859456063556137}}

In [13]:
predyction[2]['Średnie dobowe zachmurzenie ogólne [oktanty]']

{'OLS': {1: 0.5866360896840348, 2: 0.3543177237384948, 3: 0.2636268837944211},
 'RANSAC': {1: -8.622468165584161,
  2: -10.608219850162623,
  3: -21.02105555608969},
 'HuberRegressor': {1: 0.5874264148507093,
  2: 0.34258909599956033,
  3: 0.23647498902545405},
 'Ridge': {1: 0.5301733605980002,
  2: 0.3587842415571977,
  3: 0.25011637326339164}}

In [14]:
def labelowanko(Cechy_predyktowane_string,ilosc_kategorii,dataset):
    max_value = dataset[Cechy_predyktowane_string].max()
    binInterval = np.arange(0, max_value, max_value/ilosc_kategorii).tolist()
    binInterval.append(max_value)
    binLabels   = range(0,len(binInterval)-1)
    zachmurzenie_catogory = pd.cut(dataset[Cechy_predyktowane_string], bins = binInterval, labels=binLabels, include_lowest=True)
    return zachmurzenie_catogory

In [15]:
def Predykcja_kategoryczna(Cechy_predyktowane_string, dataset, ilosc_dni,ilosc_kategorii):
    lista_modeli = []
    lista_nazw = []
    lista_score = []
    lista_matrix = []
    lista_df_test_predykcje = []
    
    
    lista_list = []
    
    
    for dzien in range(1,ilosc_dni+1):
        lista = []
        max_val = dataset[Cechy_predyktowane_string].max()
        lista = labelowanko(Cechy_predyktowane_string,ilosc_kategorii,dataset).shift(-dzien)[:-dzien].tolist()
        mniejszy_dataset = dataset[:-dzien] #sciaganie ostatnich X wartosci, bo dla nich nie mamy etykiet.
        
        values = pd.DataFrame(lista, columns = ["Kategorie spredytkowane"])
        train,test, etykiety_train, etykiety_test  = train_test_split(mniejszy_dataset,values, test_size=0.3, random_state=12)
        
        
        log = LogisticRegression(random_state=0,solver='lbfgs')
        log.max_iter = 1000
        log.fit(train, etykiety_train)
        wynik_log = log.predict(test)
        log_df = pd.DataFrame(wynik_log)
        log_df = pd.concat([log_df, etykiety_test], axis=1, join='inner')
        log_df.columns = ["Kategorie rzeczywiste", "Kategorie spredytkowane"]
        
              
        lista_modeli.append(log)
        lista_nazw.append("LogisticRegression")
        lista_score.append(log.score(test,etykiety_test))
        lista_matrix.append(confusion_matrix(etykiety_test, log.predict(test)))
        lista_df_test_predykcje.append(log_df)
        
        
          
        clf = AdaBoostClassifier(n_estimators=100, random_state=0)
        clf.fit(train,etykiety_train)
        wynik_clf = clf.predict(test)
        clf_df = pd.DataFrame(wynik_log)
        clf_df = pd.concat([clf_df, etykiety_test], axis=1, join='inner')
        clf_df.columns = ["Kategorie rzeczywiste", "Kategorie spredytkowane"]
        
              
        lista_modeli.append(clf)
        lista_nazw.append("Adaboost")
        lista_score.append(log.score(test,etykiety_test))
        lista_matrix.append(confusion_matrix(etykiety_test, clf.predict(test)))
        lista_df_test_predykcje.append(clf_df)
        
        
        
        rfc = RandomForestClassifier(max_depth=50, random_state=0)
        rfc.fit(train, etykiety_train)
        wynik_rfc = rfc.predict(test)
        rfc_df = pd.DataFrame(wynik_rfc)
        rfc_df = pd.concat([rfc_df, etykiety_test], axis=1, join='inner')
        rfc_df.columns = ["Kategorie rzeczywiste", "Kategorie spredytkowane"]
        
        
        lista_modeli.append(rfc)
        lista_nazw.append("RandomForestClassifier")
        lista_score.append(rfc.score(test,etykiety_test))
        lista_matrix.append(confusion_matrix(etykiety_test, rfc.predict(test)))
        lista_df_test_predykcje.append(rfc_df)
        
        
    
        SVM = SVC(gamma=1)
        SVM.fit(train, etykiety_train)
        wynik_SVM = SVM.predict(test)
        SVM_df = pd.DataFrame(wynik_SVM)
        SVM_df = pd.concat([SVM_df, etykiety_test], axis=1, join='inner')
        SVM_df.columns = ["Kategorie rzeczywiste", "Kategorie spredytkowane"]
        
        
        lista_modeli.append(SVM)
        lista_nazw.append("SVM")
        lista_score.append(SVM.score(test,etykiety_test))
        lista_matrix.append(confusion_matrix(etykiety_test, SVM.predict(test)))
        lista_df_test_predykcje.append(SVM_df)
    
    
    
    return lista_modeli, lista_nazw, lista_score, lista_matrix, lista_df_test_predykcje

In [16]:
#Model przewidywania opadów kategoryczny
ModeleO,nazwyO,scoryO,matrixyO,test_predykcjeO = Predykcja_kategoryczna("Suma dobowa opadu [mm]", dataset, 7,3)
#Model nasłonecznienie kategoryczny
ModeleU,nazwyU,scoryU,matrixyU,test_predykcjeU = Predykcja_kategoryczna("Usłonecznienie [godziny]", dataset, 7,3)
#Model przewidywania wiatru kategoryczny
ModeleW,nazwyW,scoryW,matrixyW,test_predykcjeW = Predykcja_kategoryczna("Średnia dobowa prędkość wiatru [m/s]", dataset, 7,3)
#Model przewidywania zachmurzenia kategoryczny
ModeleZ,nazwyZ,scoryZ,matrixyZ,test_predykcjeZ = Predykcja_kategoryczna("Średnie dobowe zachmurzenie ogólne [oktanty]", dataset, 7,3)

F:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_o

F:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_o

F:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_o

F:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for exam

In [17]:
def ScoreMacierzy(matrix):
    difference = 10
    score = 0
    shape = matrix.shape
    for i in range(shape[0]):
        for j in range(shape[1]):
            point = (- abs(i-j))
            elScore = 10 if point == 0 else point * difference
            score = score + matrix[i][j] * elScore
    return score

def ChooseBestOne(matrix):
    scores = []
    scores.append([ScoreMacierzy(matrix) for matrix in matrixyO[0:4]])
    scores.append([ScoreMacierzy(matrix) for matrix in matrixyO[4:8]])
    scores.append([ScoreMacierzy(matrix) for matrix in matrixyO[8:12]])
    scores.append([ScoreMacierzy(matrix) for matrix in matrixyO[12:16]])
    scores.append([ScoreMacierzy(matrix) for matrix in matrixyO[16:20]])
    scores.append([ScoreMacierzy(matrix) for matrix in matrixyO[20:24]])
    scores.append([ScoreMacierzy(matrix) for matrix in matrixyO[24:28]])
    sums = np.sum(scores,axis=0)
    return np.where(sums == np.amax(sums))[0][0]

In [18]:
scores = []
scores.append([ScoreMacierzy(matrix) for matrix in matrixyU[0:4]])
scores.append([ScoreMacierzy(matrix) for matrix in matrixyU[4:8]])
scores.append([ScoreMacierzy(matrix) for matrix in matrixyU[8:12]])
scores.append([ScoreMacierzy(matrix) for matrix in matrixyU[12:16]])
scores.append([ScoreMacierzy(matrix) for matrix in matrixyU[16:20]])
scores.append([ScoreMacierzy(matrix) for matrix in matrixyU[20:24]])
scores.append([ScoreMacierzy(matrix) for matrix in matrixyU[24:28]])
scores

[[7170, 6280, 7610, 5630],
 [4940, 3950, 5340, 5970],
 [3930, 2690, 3370, 2970],
 [3250, 770, 3750, 3820],
 [2410, 390, 2490, 1870],
 [2480, 1030, 3040, 2770],
 [2560, 210, 2540, 2130]]

In [19]:
scores = []
scores.append([ScoreMacierzy(matrix) for matrix in matrixyW[0:4]])
scores.append([ScoreMacierzy(matrix) for matrix in matrixyW[4:8]])
scores.append([ScoreMacierzy(matrix) for matrix in matrixyW[8:12]])
scores.append([ScoreMacierzy(matrix) for matrix in matrixyW[12:16]])
scores.append([ScoreMacierzy(matrix) for matrix in matrixyW[16:20]])
scores.append([ScoreMacierzy(matrix) for matrix in matrixyW[20:24]])
scores.append([ScoreMacierzy(matrix) for matrix in matrixyW[24:28]])
scores

[[5170, 3930, 5490, 4400],
 [5270, 4400, 5330, 4880],
 [4650, 3920, 4490, 4260],
 [4690, 4390, 5050, 4190],
 [4040, 4170, 4380, 3870],
 [4350, 4420, 4240, 4320],
 [4580, 4260, 4530, 4540]]

In [20]:
scores = []
scores.append([ScoreMacierzy(matrix) for matrix in matrixyW[0:4]])
scores.append([ScoreMacierzy(matrix) for matrix in matrixyW[4:8]])
scores.append([ScoreMacierzy(matrix) for matrix in matrixyW[8:12]])
scores.append([ScoreMacierzy(matrix) for matrix in matrixyW[12:16]])
scores.append([ScoreMacierzy(matrix) for matrix in matrixyW[16:20]])
scores.append([ScoreMacierzy(matrix) for matrix in matrixyW[20:24]])
scores.append([ScoreMacierzy(matrix) for matrix in matrixyW[24:28]])
scores

[[5170, 3930, 5490, 4400],
 [5270, 4400, 5330, 4880],
 [4650, 3920, 4490, 4260],
 [4690, 4390, 5050, 4190],
 [4040, 4170, 4380, 3870],
 [4350, 4420, 4240, 4320],
 [4580, 4260, 4530, 4540]]

In [21]:
print(ChooseBestOne(matrixyO))  #Opad
print(ChooseBestOne(matrixyU))  # Usłonecznienie
print(ChooseBestOne(matrixyW))  # Wiatr
print(ChooseBestOne(matrixyZ))  #Zachmurzenie

2
2
2
2


In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification

In [23]:
scoryZ

[0.7281045751633987,
 0.7281045751633987,
 0.7372549019607844,
 0.6549019607843137,
 0.658600392413342,
 0.658600392413342,
 0.6520601700457815,
 0.6742969260954872,
 0.5958142576847613,
 0.5958142576847613,
 0.6115107913669064,
 0.5735775016350556,
 0.5951602354480052,
 0.5951602354480052,
 0.6173969914977109,
 0.6213211249182472,
 0.5896596858638743,
 0.5896596858638743,
 0.5857329842931938,
 0.5837696335078534,
 0.6145287958115183,
 0.6145287958115183,
 0.6106020942408377,
 0.618455497382199,
 0.6053664921465969,
 0.6053664921465969,
 0.5988219895287958,
 0.587696335078534]

### Optymalizacja używając GridSearch nasłonecznienie Tak/Nie

In [24]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_curve, precision_recall_curve, auc, \
make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix

In [25]:
clf = RandomForestClassifier(n_jobs=-1)

param_grid = {
    'min_samples_split': [3, 5, 10], 
    'n_estimators' : [100, 300],
    'max_depth': [3, 5, 15, 25],
    'max_features': [3, 5, 10, 20]
}

In [26]:
scorers = {
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score),
    'accuracy_score': make_scorer(accuracy_score)
}

In [27]:
max_value = dataset['Usłonecznienie [godziny]'].max()
binInterval = np.arange(0, max_value, max_value/2).tolist()
binInterval.append(max_value)
binLabels   = range(0,len(binInterval)-1)
targets = pd.cut(dataset['Usłonecznienie [godziny]'], bins = binInterval, labels=binLabels, include_lowest=True)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(dataset[:-1], targets.shift(-1)[:-1], stratify=targets.shift(-1)[:-1])

In [29]:
print('y_train class distribution')
print(y_train.value_counts(normalize=True))

print('y_test class distribution')
print(y_test.value_counts(normalize=True))

y_train class distribution
0    0.677656
1    0.322344
Name: Usłonecznienie [godziny], dtype: float64
y_test class distribution
0    0.677647
1    0.322353
Name: Usłonecznienie [godziny], dtype: float64


In [30]:
def grid_search_wrapper(scorers, refit_score='precision_score'):
    """
    fits a GridSearchCV classifier using refit_score for optimization
    prints classifier performance metrics
    """
    skf = StratifiedKFold(n_splits=10)
    grid_search = GridSearchCV(clf, param_grid, scoring=scorers, refit=refit_score,
                           cv=skf, return_train_score=True, n_jobs=-1)
    grid_search.fit(X_train.values, y_train.values)

    # make the predictions
    y_pred = grid_search.predict(X_test.values)

    print('Best params for {}'.format(refit_score))
    print(grid_search.best_params_)
    print('\nConfusion matrix of Random Forest optimized for {} on the test data:'.format(refit_score))
    print(pd.DataFrame(confusion_matrix(y_test, y_pred),
                 columns=['pred_neg', 'pred_pos'], index=['neg', 'pos']))
    return grid_search

In [31]:
grid_search_clf = grid_search_wrapper(scorers)

Best params for precision_score
{'max_depth': 3, 'max_features': 3, 'min_samples_split': 5, 'n_estimators': 100}

Confusion matrix of Random Forest optimized for precision_score on the test data:
     pred_neg  pred_pos
neg       813        51
pos       195       216


In [32]:
results = pd.DataFrame(grid_search_clf.cv_results_)
results.sort_values(by='mean_test_precision_score', ascending=False, inplace = True)
results[['mean_test_precision_score', 'mean_test_recall_score', 'mean_test_accuracy_score',
         'param_max_depth', 'param_max_features', 'param_min_samples_split',
         'param_n_estimators']].head()

,mean_test_precision_score,mean_test_recall_score,mean_test_accuracy_score,param_max_depth,param_max_features,param_min_samples_split,param_n_estimators
2,0.821916,0.543798,0.814492,3,3,5,100
3,0.820562,0.550308,0.815799,3,3,5,300
5,0.819763,0.553547,0.816321,3,3,10,300
0,0.819217,0.538133,0.811874,3,3,3,100
1,0.817612,0.551947,0.815278,3,3,3,300


In [33]:
results = pd.DataFrame(grid_search_clf.cv_results_)
results = results.sort_values(by='mean_test_recall_score', ascending=False)
results[['mean_test_precision_score', 'mean_test_recall_score', 'mean_test_accuracy_score',
         'param_max_depth', 'param_max_features', 'param_min_samples_split',
         'param_n_estimators']].head()


,mean_test_precision_score,mean_test_recall_score,mean_test_accuracy_score,param_max_depth,param_max_features,param_min_samples_split,param_n_estimators
69,0.789976,0.766214,0.858707,15,20,5,300
71,0.791007,0.763788,0.858445,15,20,10,300
68,0.788024,0.762962,0.856875,15,20,5,100
66,0.785149,0.761349,0.855569,15,20,3,100
91,0.788163,0.761349,0.856616,25,20,3,300


In [34]:
params_best_sun = results[['mean_test_precision_score', 'mean_test_recall_score', 'mean_test_accuracy_score',
         'param_max_depth', 'param_max_features', 'param_min_samples_split',
         'param_n_estimators']].head(1)
params_best_sun

,mean_test_precision_score,mean_test_recall_score,mean_test_accuracy_score,param_max_depth,param_max_features,param_min_samples_split,param_n_estimators
69,0.789976,0.766214,0.858707,15,20,5,300



## Optimising srednia temperatura dobowa Ridge Regression

In [35]:
model = make_pipeline(PolynomialFeatures(2), Ridge())

In [36]:
X_train, X_test, y_train, y_test = train_test_split(dataset[:-1], dataset["Średnia temperatura dobowa [°C]"].shift(-1)[:-1], stratify=targets.shift(-1)[:-1])

In [37]:
model.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'polynomialfeatures', 'ridge', 'polynomialfeatures__degree', 'polynomialfeatures__include_bias', 'polynomialfeatures__interaction_only', 'polynomialfeatures__order', 'ridge__alpha', 'ridge__copy_X', 'ridge__fit_intercept', 'ridge__max_iter', 'ridge__normalize', 'ridge__random_state', 'ridge__solver', 'ridge__tol'])

In [38]:
# find optimal alpha with grid search
alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
param_grid = dict(alpha=alpha)
grid = GridSearchCV(estimator=Ridge(), param_grid=param_grid, scoring='r2', verbose=1, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Best Score:  0.9427261031057685
Best Params:  {'alpha': 0.01}


[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    0.1s finished


In [39]:
params_best_temp = grid_result.best_params_
params_best_temp

{'alpha': 0.01}

## Odnalezienie najlepszego klasyfikatora 2 

In [40]:
class EstimatorSelectionHelper:

    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv=3, n_jobs=-1, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X,y)
            self.grid_searches[key] = gs    

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series({**params,**d})

        rows = []
        for k in self.grid_searches:
            print(k)
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]        
                scores.append(r.reshape(len(params),1))

            all_scores = np.hstack(scores)
            for p, s in zip(params,all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns]

In [41]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [42]:
models1 = {
    'ExtraTreesClassifier': ExtraTreesClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    'SVC': SVC()
}


In [43]:
params1 = {
    'ExtraTreesClassifier': { 'n_estimators': [16, 32] },
    'RandomForestClassifier': { 'n_estimators': [16, 32] },
    'AdaBoostClassifier':  { 'n_estimators': [16, 32] },
    'GradientBoostingClassifier': { 'n_estimators': [16, 32], 'learning_rate': [0.8, 1.0] },
    'SVC': [
        {'kernel': ['linear'], 'C': [1, 10]},
        {'kernel': ['rbf'], 'C': [1, 10], 'gamma': [0.001, 0.0001]},
    ]
}

In [44]:
max_value = dataset['Suma dobowa opadu [mm]'].max()
binInterval = np.arange(0, max_value, max_value/3).tolist()
binInterval.append(max_value)
binLabels   = range(0,len(binInterval)-1)
targets = pd.cut(dataset['Suma dobowa opadu [mm]'], bins = binInterval, labels=binLabels, include_lowest=True)

In [45]:
X_train, X_test, y_train, y_test = train_test_split(dataset[:-1], targets.shift(-1)[:-1], stratify=targets.shift(-1)[:-1])

In [46]:
helper1 = EstimatorSelectionHelper(models1, params1)

In [47]:
helper1.fit(X_train, y_train, scoring='f1_macro', n_jobs=-1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Running GridSearchCV for ExtraTreesClassifier.
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Running GridSearchCV for RandomForestClassifier.
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Running GridSearchCV for GradientBoostingClassifier.
Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done   2 out of  12 | elapsed:    0.6s remaining:    3.3s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    1.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Running GridSearchCV for SVC.
Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Done  14 out of  18 | elapsed:    2.8s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    2.9s finished


In [48]:
helper1.score_summary(sort_by='max_score')

ExtraTreesClassifier
RandomForestClassifier
AdaBoostClassifier
GradientBoostingClassifier
SVC


,estimator,min_score,mean_score,max_score,std_score,n_estimators,learning_rate,C,kernel,gamma
5,AdaBoostClassifier,0.434104,0.453211,0.480303,0.019688,32,NaN,NaN,NaN,NaN
2,RandomForestClassifier,0.435506,0.45495,0.477882,0.017475,16,NaN,NaN,NaN,NaN
11,SVC,0.447959,0.459414,0.469569,0.00887031,NaN,NaN,10,linear,NaN
6,GradientBoostingClassifier,0.433884,0.451262,0.468997,0.014337,16,0.8,NaN,NaN,NaN
10,SVC,0.44815,0.45945,0.468936,0.00858222,NaN,NaN,1,linear,NaN
1,ExtraTreesClassifier,0.436823,0.45469,0.466727,0.0128855,32,NaN,NaN,NaN,NaN
4,AdaBoostClassifier,0.440908,0.456778,0.46657,0.0113241,16,NaN,NaN,NaN,NaN
8,GradientBoostingClassifier,0.433128,0.447532,0.465952,0.0136978,16,1,NaN,NaN,NaN
7,GradientBoostingClassifier,0.454486,0.458872,0.465927,0.00503788,32,0.8,NaN,NaN,NaN
0,ExtraTreesClassifier,0.424738,0.440269,0.456052,0.012785,16,NaN,NaN,NaN,NaN


In [49]:
params_best_rain = helper1.score_summary(sort_by='max_score').head(1)
params_best_rain

ExtraTreesClassifier
RandomForestClassifier
AdaBoostClassifier
GradientBoostingClassifier
SVC


,estimator,min_score,mean_score,max_score,std_score,n_estimators,learning_rate,C,kernel,gamma
5,AdaBoostClassifier,0.434104,0.453211,0.480303,0.019688,32,NaN,NaN,NaN,NaN


In [50]:
max_value = dataset['Średnie dobowe zachmurzenie ogólne [oktanty]'].max()
binInterval = np.arange(0, max_value, max_value/3).tolist()
binInterval.append(max_value)
binLabels   = range(0,len(binInterval)-1)
targets = pd.cut(dataset['Średnie dobowe zachmurzenie ogólne [oktanty]'], bins = binInterval, labels=binLabels, include_lowest=True)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(dataset[:-1], targets.shift(-1)[:-1], stratify=targets.shift(-1)[:-1])

In [52]:
helper2 = EstimatorSelectionHelper(models1, params1)

In [53]:
helper2.fit(X_train, y_train, scoring='f1_macro', n_jobs=-1)

Running GridSearchCV for ExtraTreesClassifier.
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Running GridSearchCV for RandomForestClassifier.
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Running GridSearchCV for AdaBoostClassifier.
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Running GridSearchCV for GradientBoostingClassifier.
Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done   2 out of  12 | elapsed:    0.6s remaining:    3.3s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    1.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Running GridSearchCV for SVC.
Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Done  14 out of  18 | elapsed:    3.3s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    3.8s finished


In [54]:
helper2.score_summary(sort_by='max_score')

ExtraTreesClassifier
RandomForestClassifier
AdaBoostClassifier
GradientBoostingClassifier
SVC


,estimator,min_score,mean_score,max_score,std_score,n_estimators,learning_rate,C,kernel,gamma
8,GradientBoostingClassifier,0.618243,0.646729,0.682058,0.0264979,16,1,NaN,NaN,NaN
5,AdaBoostClassifier,0.623195,0.638866,0.66158,0.0164431,32,NaN,NaN,NaN,NaN
6,GradientBoostingClassifier,0.626584,0.643398,0.659553,0.0134677,16,0.8,NaN,NaN,NaN
11,SVC,0.643568,0.649916,0.659515,0.00690456,NaN,NaN,10,linear,NaN
7,GradientBoostingClassifier,0.623041,0.640642,0.658216,0.0143604,32,0.8,NaN,NaN,NaN
4,AdaBoostClassifier,0.600663,0.62503,0.655251,0.0226662,16,NaN,NaN,NaN,NaN
10,SVC,0.63571,0.645982,0.653649,0.00755144,NaN,NaN,1,linear,NaN
3,RandomForestClassifier,0.644873,0.648234,0.650172,0.00238593,32,NaN,NaN,NaN,NaN
9,GradientBoostingClassifier,0.607151,0.633717,0.650032,0.0189472,32,1,NaN,NaN,NaN
2,RandomForestClassifier,0.623588,0.632076,0.63805,0.00616609,16,NaN,NaN,NaN,NaN


In [55]:
params_best_clouds = helper2.score_summary(sort_by='max_score').head(1)
params_best_clouds

ExtraTreesClassifier
RandomForestClassifier
AdaBoostClassifier
GradientBoostingClassifier
SVC


,estimator,min_score,mean_score,max_score,std_score,n_estimators,learning_rate,C,kernel,gamma
8,GradientBoostingClassifier,0.618243,0.646729,0.682058,0.0264979,16,1,NaN,NaN,NaN


In [56]:
max_value = dataset['Średnia dobowe ciśnienie na poziomie stacji [hPa]'].max()
binInterval = np.arange(0, max_value, max_value/3).tolist()
binInterval.append(max_value)
binLabels   = range(0,len(binInterval)-1)
targets = pd.cut(dataset['Średnia dobowe ciśnienie na poziomie stacji [hPa]'], bins = binInterval, labels=binLabels, include_lowest=True)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(dataset[:-1], targets.shift(-1)[:-1], stratify=targets.shift(-1)[:-1])

In [58]:
helper3 = EstimatorSelectionHelper(models1, params1)

In [59]:
helper3.fit(X_train, y_train, scoring='f1_macro', n_jobs=-1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Running GridSearchCV for ExtraTreesClassifier.
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Running GridSearchCV for RandomForestClassifier.
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Running GridSearchCV for GradientBoostingClassifier.
Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done   2 out of  12 | elapsed:    0.6s remaining:    3.4s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    1.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Running GridSearchCV for SVC.
Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Done  14 out of  18 | elapsed:    3.8s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    4.1s finished


In [60]:
helper3.score_summary(sort_by='max_score')

ExtraTreesClassifier
RandomForestClassifier
AdaBoostClassifier
GradientBoostingClassifier
SVC


,estimator,min_score,mean_score,max_score,std_score,n_estimators,learning_rate,C,kernel,gamma
6,GradientBoostingClassifier,0.709582,0.72905,0.749948,0.0165102,16,0.8,NaN,NaN,NaN
7,GradientBoostingClassifier,0.699584,0.724782,0.746652,0.0193588,32,0.8,NaN,NaN,NaN
8,GradientBoostingClassifier,0.700586,0.715361,0.73492,0.0144189,16,1,NaN,NaN,NaN
9,GradientBoostingClassifier,0.689,0.707823,0.721326,0.013723,32,1,NaN,NaN,NaN
5,AdaBoostClassifier,0.696982,0.706215,0.720595,0.0103044,32,NaN,NaN,NaN,NaN
4,AdaBoostClassifier,0.693138,0.700332,0.710705,0.00751566,16,NaN,NaN,NaN,NaN
3,RandomForestClassifier,0.621128,0.642539,0.665733,0.0182536,32,NaN,NaN,NaN,NaN
2,RandomForestClassifier,0.631964,0.6407,0.654865,0.0101073,16,NaN,NaN,NaN,NaN
0,ExtraTreesClassifier,0.495706,0.523284,0.539159,0.0195748,16,NaN,NaN,NaN,NaN
1,ExtraTreesClassifier,0.527368,0.529869,0.532547,0.00211786,32,NaN,NaN,NaN,NaN


In [61]:
params_best_pressure = helper3.score_summary(sort_by='max_score').head(1)
params_best_pressure

ExtraTreesClassifier
RandomForestClassifier
AdaBoostClassifier
GradientBoostingClassifier
SVC


,estimator,min_score,mean_score,max_score,std_score,n_estimators,learning_rate,C,kernel,gamma
6,GradientBoostingClassifier,0.709582,0.72905,0.749948,0.0165102,16,0.8,NaN,NaN,NaN


In [62]:
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV


lasso_pipeline = make_pipeline(LassoCV())

rf_pipeline = make_pipeline(RandomForestRegressor(random_state=42))

gradient_pipeline = make_pipeline(HistGradientBoostingRegressor(random_state=0))

estimators = [('Random Forest', rf_pipeline),
              ('Lasso', lasso_pipeline),
              ('Gradient Boosting', gradient_pipeline)]

stacking_regressor = StackingRegressor(estimators=estimators,
                                       final_estimator=RidgeCV())

In [63]:
X_train, X_test, y_train, y_test = train_test_split(dataset[:-1], dataset['Usłonecznienie [godziny]'].shift(-1)[:-1],\
                                                    stratify=dataset['Usłonecznienie [godziny]'].shift(-1)[:-1])

In [64]:
%%time

from sklearn.model_selection import cross_validate, cross_val_predict

score = cross_validate(stacking_regressor, X_train, y_train,
                           scoring=['r2', 'neg_mean_absolute_error'],
                           n_jobs=-1, verbose=0)

y_pred = cross_val_predict(stacking_regressor, X_test, y_test, n_jobs=-1, verbose=0)

Wall time: 49.1 s


In [65]:
score

{'fit_time': array([30.65536785, 30.83987999, 30.98818326, 30.38405466, 30.52293825]),
 'score_time': array([0.04563236, 0.03720093, 0.02777624, 0.05158448, 0.0610044 ]),
 'test_r2': array([0.67943292, 0.70150265, 0.69576273, 0.71306299, 0.68420408]),
 'test_neg_mean_absolute_error': array([-0.11977164, -0.12032355, -0.12111798, -0.12102288, -0.12426645])}

In [66]:
y_pred

array([0.47097323, 0.16628265, 0.06328488, ..., 0.47845263, 0.07153413,
       0.11275665])

In [67]:
print("Accuracy: %0.2f (+/- %0.2f)" % (y_pred.mean(), y_pred.std() * 2))

Accuracy: 0.33 (+/- 0.48)


In [68]:
dataset = pd.read_json('dataset')

# Załadowanie datasetu walidacyjnego 2020

In [69]:
df_sdt1 = pd.read_csv("data_2020/s_d_t_01_2020.csv", index_col=None, header=0, encoding='latin1')
df_sdt1.columns = ['Kod stacji', 'Nazwa stacji', 'Rok', 'Miesiac', 'Dzien', 'Średnie dobowe zachmurzenie ogólne [oktanty]', 'Status pomiaru NOS ', 'Średnia dobowa prędkość wiatru [m/s]', 'Status pomiaru FWS ', 
              'Średnia dobowa temperatura [°C]', 'Status pomiaru TEMP', 'Średnia dobowe ciśnienie pary wodnej [hPa]', 'Status pomiaru CPW', 'Średnia dobowa wilgotność względna [%]', 'Status pomiaru WLGS',
              'Średnia dobowe ciśnienie na poziomie stacji [hPa]', 'Status pomiaru PPPS ', 'Średnie dobowe ciśnienie na pozimie morza [hPa]', 'Status pomiaru PPPM', 'Suma opadu dzień  [mm]', 'Status pomiaru WODZ', 
              'Suma opadu noc   [mm] ','Status pomiaru WONO']

df_sdt2 = pd.read_csv("data_2020/s_d_t_02_2020.csv", index_col=None, header=0, encoding='latin1')
df_sdt2.columns = ['Kod stacji', 'Nazwa stacji', 'Rok', 'Miesiac', 'Dzien', 'Średnie dobowe zachmurzenie ogólne [oktanty]', 'Status pomiaru NOS ', 'Średnia dobowa prędkość wiatru [m/s]', 'Status pomiaru FWS ', 
              'Średnia dobowa temperatura [°C]', 'Status pomiaru TEMP', 'Średnia dobowe ciśnienie pary wodnej [hPa]', 'Status pomiaru CPW', 'Średnia dobowa wilgotność względna [%]', 'Status pomiaru WLGS',
              'Średnia dobowe ciśnienie na poziomie stacji [hPa]', 'Status pomiaru PPPS ', 'Średnie dobowe ciśnienie na pozimie morza [hPa]', 'Status pomiaru PPPM', 'Suma opadu dzień  [mm]', 'Status pomiaru WODZ', 
              'Suma opadu noc   [mm] ','Status pomiaru WONO']

df_sdt3 = pd.read_csv("data_2020/s_d_t_03_2020.csv", index_col=None, header=0, encoding='latin1')
df_sdt3.columns = ['Kod stacji', 'Nazwa stacji', 'Rok', 'Miesiac', 'Dzien', 'Średnie dobowe zachmurzenie ogólne [oktanty]', 'Status pomiaru NOS ', 'Średnia dobowa prędkość wiatru [m/s]', 'Status pomiaru FWS ', 
              'Średnia dobowa temperatura [°C]', 'Status pomiaru TEMP', 'Średnia dobowe ciśnienie pary wodnej [hPa]', 'Status pomiaru CPW', 'Średnia dobowa wilgotność względna [%]', 'Status pomiaru WLGS',
              'Średnia dobowe ciśnienie na poziomie stacji [hPa]', 'Status pomiaru PPPS ', 'Średnie dobowe ciśnienie na pozimie morza [hPa]', 'Status pomiaru PPPM', 'Suma opadu dzień  [mm]', 'Status pomiaru WODZ', 
              'Suma opadu noc   [mm] ','Status pomiaru WONO']



df_sd1 = pd.read_csv("data_2020/s_d_01_2020.csv", index_col=None, header=0, encoding='latin1')
df_sd1.columns = ['Kod stacji', 'Nazwa stacji', 'Rok', 'Miesiac', 'Dzien', 'Maksymalna temperatura dobowa [°C]', 'Status pomiaru TMAX', 'Minimalna temperatura dobowa [°C]', 'Status pomiaru TMIN', 
              'Średnia temperatura dobowa [°C]', 'Status pomiaru STD ', 'Temperatura minimalna przy gruncie [°C] ', 'Status pomiaru TMNG  ', 'Suma dobowa opadu [mm]', 'Status pomiaru SMDB',
              'Rodzaj opadu [S/W/ ] ', 'Wysokość pokrywy śnieżnej [cm] ', 'Status pomiaru PKSN ', 'Równoważnik wodny śniegu  [mm/cm]', 'Status pomiaru RWSN ', 'Usłonecznienie [godziny]', 
              'Status pomiaru USL','Czas trwania opadu deszczu [godziny]', 'Status pomiaru DESZ', 'Czas trwania opadu śniegu [godziny]', 'Status pomiaru SNEG', 'Czas trwania opadu deszczu ze śniegiem [godziny]'
              , 'Status pomiaru DISN  ', 'Czas trwania gradu [godziny] ', 'Status pomiaru GRAD', 'Czas trwania mgły [godziny]', 'Status pomiaru MGLA', 'Czas trwania zamglenia  [godziny]', 'Status pomiaru ZMGL',
              'Czas trwania sadzi [godziny]', 'Status pomiaru SADZ', 'Czas trwania gołoledzi [godziny]', 'Status pomiaru GOLO ', 'Czas trwania zamieci śnieżnej niskiej [godziny]', 'Status pomiaru ZMNI', 'Czas trwania zamieci śnieżnej wysokiej [godziny]',
              'Status pomiaru ZMWS', 'Czas trwania zmętnienia [godziny]', 'Status pomiaru ZMET', 'Czas trwania wiatru >=10m/s [godziny] ', 'Status pomiaru FF10', 'Czas trwania wiatru >15m/s [godziny]', 'Status pomiaru FF15'
              , 'Czas trwania burzy  [godziny]', 'Status pomiaru BRZA ', 'Czas trwania rosy  [godziny]', 'Status pomiaru ROSA'
             , 'Czas trwania szronu [godziny]', 'Status pomiaru SZRO', 'Wystąpienie pokrywy śnieżnej  [0/1]', 'Status pomiaru DZPS', 'Wystąpienie błyskawicy  [0/1]', 'Status pomiaru DZBL ', 'Stan gruntu [Z/R]',
              'Izoterma dolna  [cm] ', 'Status pomiaru IZD', 'Izoterma górna [cm] ', 'Status pomiaru IZG ', 'Aktynometria  [J/cm2]', 'Status pomiaru AKTN ']

df_sd2 = pd.read_csv("data_2020/s_d_02_2020.csv" , index_col=None, header=0, encoding='latin1')
df_sd2.columns = ['Kod stacji', 'Nazwa stacji', 'Rok', 'Miesiac', 'Dzien', 'Maksymalna temperatura dobowa [°C]', 'Status pomiaru TMAX', 'Minimalna temperatura dobowa [°C]', 'Status pomiaru TMIN', 
              'Średnia temperatura dobowa [°C]', 'Status pomiaru STD ', 'Temperatura minimalna przy gruncie [°C] ', 'Status pomiaru TMNG  ', 'Suma dobowa opadu [mm]', 'Status pomiaru SMDB',
              'Rodzaj opadu [S/W/ ] ', 'Wysokość pokrywy śnieżnej [cm] ', 'Status pomiaru PKSN ', 'Równoważnik wodny śniegu  [mm/cm]', 'Status pomiaru RWSN ', 'Usłonecznienie [godziny]', 
              'Status pomiaru USL','Czas trwania opadu deszczu [godziny]', 'Status pomiaru DESZ', 'Czas trwania opadu śniegu [godziny]', 'Status pomiaru SNEG', 'Czas trwania opadu deszczu ze śniegiem [godziny]'
              , 'Status pomiaru DISN  ', 'Czas trwania gradu [godziny] ', 'Status pomiaru GRAD', 'Czas trwania mgły [godziny]', 'Status pomiaru MGLA', 'Czas trwania zamglenia  [godziny]', 'Status pomiaru ZMGL',
              'Czas trwania sadzi [godziny]', 'Status pomiaru SADZ', 'Czas trwania gołoledzi [godziny]', 'Status pomiaru GOLO ', 'Czas trwania zamieci śnieżnej niskiej [godziny]', 'Status pomiaru ZMNI', 'Czas trwania zamieci śnieżnej wysokiej [godziny]',
              'Status pomiaru ZMWS', 'Czas trwania zmętnienia [godziny]', 'Status pomiaru ZMET', 'Czas trwania wiatru >=10m/s [godziny] ', 'Status pomiaru FF10', 'Czas trwania wiatru >15m/s [godziny]', 'Status pomiaru FF15'
              , 'Czas trwania burzy  [godziny]', 'Status pomiaru BRZA ', 'Czas trwania rosy  [godziny]', 'Status pomiaru ROSA'
             , 'Czas trwania szronu [godziny]', 'Status pomiaru SZRO', 'Wystąpienie pokrywy śnieżnej  [0/1]', 'Status pomiaru DZPS', 'Wystąpienie błyskawicy  [0/1]', 'Status pomiaru DZBL ', 'Stan gruntu [Z/R]',
              'Izoterma dolna  [cm] ', 'Status pomiaru IZD', 'Izoterma górna [cm] ', 'Status pomiaru IZG ', 'Aktynometria  [J/cm2]', 'Status pomiaru AKTN ']

df_sd3 = pd.read_csv("data_2020/s_d_03_2020.csv" , index_col=None, header=0, encoding='latin1')
df_sd3.columns = ['Kod stacji', 'Nazwa stacji', 'Rok', 'Miesiac', 'Dzien', 'Maksymalna temperatura dobowa [°C]', 'Status pomiaru TMAX', 'Minimalna temperatura dobowa [°C]', 'Status pomiaru TMIN', 
              'Średnia temperatura dobowa [°C]', 'Status pomiaru STD ', 'Temperatura minimalna przy gruncie [°C] ', 'Status pomiaru TMNG  ', 'Suma dobowa opadu [mm]', 'Status pomiaru SMDB',
              'Rodzaj opadu [S/W/ ] ', 'Wysokość pokrywy śnieżnej [cm] ', 'Status pomiaru PKSN ', 'Równoważnik wodny śniegu  [mm/cm]', 'Status pomiaru RWSN ', 'Usłonecznienie [godziny]', 
              'Status pomiaru USL','Czas trwania opadu deszczu [godziny]', 'Status pomiaru DESZ', 'Czas trwania opadu śniegu [godziny]', 'Status pomiaru SNEG', 'Czas trwania opadu deszczu ze śniegiem [godziny]'
              , 'Status pomiaru DISN  ', 'Czas trwania gradu [godziny] ', 'Status pomiaru GRAD', 'Czas trwania mgły [godziny]', 'Status pomiaru MGLA', 'Czas trwania zamglenia  [godziny]', 'Status pomiaru ZMGL',
              'Czas trwania sadzi [godziny]', 'Status pomiaru SADZ', 'Czas trwania gołoledzi [godziny]', 'Status pomiaru GOLO ', 'Czas trwania zamieci śnieżnej niskiej [godziny]', 'Status pomiaru ZMNI', 'Czas trwania zamieci śnieżnej wysokiej [godziny]',
              'Status pomiaru ZMWS', 'Czas trwania zmętnienia [godziny]', 'Status pomiaru ZMET', 'Czas trwania wiatru >=10m/s [godziny] ', 'Status pomiaru FF10', 'Czas trwania wiatru >15m/s [godziny]', 'Status pomiaru FF15'
              , 'Czas trwania burzy  [godziny]', 'Status pomiaru BRZA ', 'Czas trwania rosy  [godziny]', 'Status pomiaru ROSA'
             , 'Czas trwania szronu [godziny]', 'Status pomiaru SZRO', 'Wystąpienie pokrywy śnieżnej  [0/1]', 'Status pomiaru DZPS', 'Wystąpienie błyskawicy  [0/1]', 'Status pomiaru DZBL ', 'Stan gruntu [Z/R]',
              'Izoterma dolna  [cm] ', 'Status pomiaru IZD', 'Izoterma górna [cm] ', 'Status pomiaru IZG ', 'Aktynometria  [J/cm2]', 'Status pomiaru AKTN ']


In [70]:
df_sdt1.loc[df_sdt1['Nazwa stacji'] ==  'GDAÑSK-WIBNO'].head(5)

,Kod stacji,Nazwa stacji,Rok,Miesiac,Dzien,Średnie dobowe zachmurzenie ogólne [oktanty],Status pomiaru NOS,Średnia dobowa prędkość wiatru [m/s],Status pomiaru FWS,Średnia dobowa temperatura [°C],Status pomiaru TEMP,Średnia dobowe ciśnienie pary wodnej [hPa],Status pomiaru CPW,Średnia dobowa wilgotność względna [%],Status pomiaru WLGS,Średnia dobowe ciśnienie na poziomie stacji [hPa],Status pomiaru PPPS,Średnie dobowe ciśnienie na pozimie morza [hPa],Status pomiaru PPPM,Suma opadu dzień [mm],Status pomiaru WODZ,Suma opadu noc [mm],Status pomiaru WONO
1704,354180155,GDAÑSK-WIBNO,2020,1,1,4.3,NaN,3.4,NaN,2.3,NaN,6.0,NaN,83.8,NaN,1025.1,NaN,1026.3,NaN,0.0,NaN,0.0,NaN
1705,354180155,GDAÑSK-WIBNO,2020,1,2,3.8,NaN,3.8,NaN,0.6,NaN,5.1,NaN,79.3,NaN,1021.4,NaN,1022.6,NaN,0.0,NaN,0.0,NaN
1706,354180155,GDAÑSK-WIBNO,2020,1,3,5.9,NaN,4.4,NaN,1.8,NaN,6.1,NaN,86.3,NaN,1013.1,NaN,1014.2,NaN,0.2,NaN,2.0,NaN
1707,354180155,GDAÑSK-WIBNO,2020,1,4,6.6,NaN,6.5,NaN,3.7,NaN,6.6,NaN,82.6,NaN,1007.6,NaN,1008.7,NaN,2.4,NaN,0.0,NaN
1708,354180155,GDAÑSK-WIBNO,2020,1,5,5.0,NaN,5.0,NaN,2.4,NaN,5.0,NaN,69.5,NaN,1024.9,NaN,1026.0,NaN,0.0,NaN,0.7,NaN


In [71]:
df_2020list = [df_sdt1,df_sdt2,df_sdt3,df_sd1,df_sd2,df_sd3]
df_lists = []

for i in df_2020list:
    df1 = i.loc[i['Nazwa stacji'] =='GDAÑSK-WIBNO']
    df_lists.append(df1)
    

In [72]:
df_sdt_list = [df_lists[2],df_lists[1],df_lists[0]]
df_sd_list = [df_lists[5],df_lists[4],df_lists[3]]

In [73]:
merged_df=[]
for i in range(len(df_sd_list)):
    merged_df.append(pd.merge(df_sdt_list[i],df_sd_list[i],how='left',left_on=["Rok",'Miesiac',"Dzien"],right_on=["Rok",'Miesiac',"Dzien"]))

In [74]:
data_df=merged_df[0]
for i in range(1,len(merged_df)):
    data_df=data_df.append(merged_df[i])

In [75]:
both = merged_df[0].append(merged_df[1])

In [76]:
data_df['data_pomiaru'] = pd.to_datetime(data_df.Rok*10000+data_df.Miesiac*100\
                                      +data_df.Dzien,format='%Y%m%d')
data_df.drop(columns = ['Rok', 'Miesiac', 'Dzien'], inplace = True)
data_df['month'] = data_df['data_pomiaru'].dt.month.astype('category')
data_df['day'] = data_df['data_pomiaru'].dt.day.astype('category')
data_df = pd.get_dummies(data_df)
data_df.drop(columns = ['data_pomiaru'], inplace = True)

In [77]:
filtered = [col for col in dataset if col.lower()[:4] != 'year']
dataset = dataset[filtered]

In [78]:
# data2020 = data_df[dataset.select_dtypes(include=['float64']).columns]

In [79]:
dataset

,Średnie dobowe zachmurzenie ogólne [oktanty],Średnia dobowa prędkość wiatru [m/s],Średnia dobowe ciśnienie pary wodnej [hPa],Średnia dobowa wilgotność względna [%],Średnia dobowe ciśnienie na poziomie stacji [hPa],Suma opadu dzień [mm],Suma opadu noc [mm],Maksymalna temperatura dobowa [°C],Minimalna temperatura dobowa [°C],Suma dobowa opadu [mm],Temperatura minimalna przy gruncie [°C],Usłonecznienie [godziny],Średnia temperatura dobowa [°C],Czas trwania opadu deszczu [godziny],Czas trwania rosy [godziny],Czas trwania zamglenia [godziny],month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31
0,7.4,3.0,7.0,91.1,1007.1,0.60,0.0,4.6,2.8,0.6,2.3,0.6,3.2,6.75,0.0,13.2,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7.4,3.8,7.6,98.6,1010.8,0.10,0.1,5.1,2.1,0.2,0.3,1.3,3.2,5.70,0.0,7.9,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7.4,4.5,7.4,90.0,1008.3,0.75,0.5,7.3,2.2,3.5,1.7,0.0,4.2,6.75,0.0,6.9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,7.3,6.1,8.3,97.5,1011.8,0.40,0.5,8.4,1.6,1.2,-0.1,0.0,4.7,5.80,0.0,2.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4.3,3.3,7.0,85.5,1009.7,0.00,0.3,7.1,3.0,0.3,1.2,4.6,4.1,0.00,0.0,8.7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5093,6.8,3.9,5.5,78.3,1035.4,0.00,0.0,3.5,0.6,0.0,-0.1,1.3,1.8,0.70,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5094,6.5,8.0,4.4,65.9,998.5,0.00,0.5,3.1,-0.9,0.6,-0.9,0.0,1.4,2.20,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5095,6.1,5.1,4.7,60.0,1019.9,0.00,0.0,5.8,0.9,0.0,0.5,0.0,3.7,1.30,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5096,6.9,6.4,6.3,77.6,993.4,0.20,0.0,5.4,1.9,0.2,0.9,0.8,4.0,2.70,0.0,0.8,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [80]:
for column in dataset.columns:
    if column not in data_df.columns:
        data_df[column] = 0
data2020 = data_df[dataset.columns]

In [81]:
data2020.reset_index(drop = True, inplace = True)

In [82]:
day_temp_series = data2020['Średnia temperatura dobowa [°C]']
#dataset_2020.drop(columns = ['Średnia dobowa temperatura [°C]'], inplace = True)
#dataset_2020.drop(columns = ['Średnie dobowe ciśnienie na pozimie morza [hPa]'], inplace = True)
#dataset_2020.drop(columns = ['Maksymalna temperatura dobowa [°C]', 'Temperatura minimalna przy gruncie [°C] ',  'Minimalna temperatura dobowa [°C]'], inplace = True)

In [83]:
data2020.to_json("dataset2020")

In [84]:
data2020

,Średnie dobowe zachmurzenie ogólne [oktanty],Średnia dobowa prędkość wiatru [m/s],Średnia dobowe ciśnienie pary wodnej [hPa],Średnia dobowa wilgotność względna [%],Średnia dobowe ciśnienie na poziomie stacji [hPa],Suma opadu dzień [mm],Suma opadu noc [mm],Maksymalna temperatura dobowa [°C],Minimalna temperatura dobowa [°C],Suma dobowa opadu [mm],Temperatura minimalna przy gruncie [°C],Usłonecznienie [godziny],Średnia temperatura dobowa [°C],Czas trwania opadu deszczu [godziny],Czas trwania rosy [godziny],Czas trwania zamglenia [godziny],month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31
0,5.5,4.8,7.6,77.0,994.9,0.0,0.0,10.9,3.1,0.0,1.2,7.7,6.9,1.6,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5.3,3.0,7.6,91.4,995.4,0.3,0.0,8.6,0.3,0.3,-1.4,1.6,4.3,3.3,0.0,5.4,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7.1,3.1,8.5,97.3,997.2,2.4,2.2,6.5,0.3,4.6,-1.1,0.0,4.9,9.8,0.0,21.9,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,5.8,4.6,7.1,80.8,1004.4,0.0,0.0,8.9,3.3,0.0,-0.3,2.5,5.1,0.4,0.0,0.6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5.8,3.4,6.6,85.5,1005.5,0.0,0.0,7.8,-0.8,0.0,-2.3,5.8,3.3,0.0,0.0,8.8,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,8.0,4.6,6.3,94.8,1011.0,0.0,0.0,2.0,-0.5,0.0,-0.9,0.0,1.0,0.0,0.0,18.7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
87,7.9,6.3,7.1,95.0,995.7,0.8,1.1,4.5,0.6,1.9,0.0,0.0,2.8,9.2,0.0,21.7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
88,6.5,5.0,6.7,89.6,992.4,0.3,0.2,4.7,1.6,0.5,-0.8,1.0,2.8,10.9,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
89,7.1,3.9,7.2,91.5,1000.4,0.5,5.8,4.3,0.5,6.3,-1.6,0.0,3.6,4.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [85]:
dataset = pd.read_json("dataset")
data2020 = pd.read_json("dataset2020")

## Nauczenie klasyfikatorów i regresorów z najlepszymi parametrami

In [86]:
params_best_sun.param_max_features.values[0]
to_predict_dataset = data2020[dataset.columns]

In [87]:
x = params_best_sun.param_max_features.values[0]
params_best_sun.param_max_features.values[0] = x if x < len(to_predict_dataset.columns) else len(to_predict_dataset.columns)

In [88]:
display(params_best_temp)
display(params_best_rain)
display(params_best_clouds)
display(params_best_pressure)

{'alpha': 0.01}

,estimator,min_score,mean_score,max_score,std_score,n_estimators,learning_rate,C,kernel,gamma
5,AdaBoostClassifier,0.434104,0.453211,0.480303,0.019688,32,NaN,NaN,NaN,NaN


,estimator,min_score,mean_score,max_score,std_score,n_estimators,learning_rate,C,kernel,gamma
8,GradientBoostingClassifier,0.618243,0.646729,0.682058,0.0264979,16,1,NaN,NaN,NaN


,estimator,min_score,mean_score,max_score,std_score,n_estimators,learning_rate,C,kernel,gamma
6,GradientBoostingClassifier,0.709582,0.72905,0.749948,0.0165102,16,0.8,NaN,NaN,NaN


In [89]:
display(params_best_sun)

,mean_test_precision_score,mean_test_recall_score,mean_test_accuracy_score,param_max_depth,param_max_features,param_min_samples_split,param_n_estimators
69,0.789976,0.766214,0.858707,15,20,5,300


In [90]:
n_estimators = params_best_sun['param_n_estimators'].values

In [91]:
params_best_sun['param_max_features'].values

array([20], dtype=object)

In [92]:
from sklearn.metrics import log_loss

In [93]:
dataset['Średnia dobowe ciśnienie na poziomie stacji [hPa]'].max()

1042.0

In [94]:
class Classifier:
    classifiers = {'ExtraTreesClassifier': ExtraTreesClassifier,
                       'RandomForestClassifier': RandomForestClassifier,
                       'AdaBoostClassifier': AdaBoostClassifier,
                       'GradientBoostingClassifier': GradientBoostingClassifier,
                       'SVC': SVC}
    
    def __init__(self, X_data, Y_data):
        self.X = X_data.copy()
        self.Y = Y_data.copy()
        self.scaler = None
        self.trained_models = {}
        self.binIntervals = None
        self.accuracyScore = None
        self.n_days = None
        self.prediction = None
        self.categorical = False
    
    def fit(self, n_days=0, n_classes=3, clf = None, params = {}):
        if clf:
            print({**params})
            self._scaleData()
            self._categorize(n_classes)
            self.categorical = True
        for i in range(1, n_days+1):
            model = self.classifiers[clf](**params) if clf \
                        else make_pipeline(MinMaxScaler(), PolynomialFeatures(2), Ridge(alpha= 0.001))
            
            X_train, X_test, y_train, y_test = train_test_split(self.X[:-i], self.Y.shift(-i)[:-i],\
                                                                stratify=self.Y.shift(-i)[:-i]) if clf\
                                                else train_test_split(self.X[:-i], self.Y.shift(-i)[:-i])
#             print(set(y_train))
            self.trained_models[i] = model.fit(X_train, y_train)
            
        accuracy = pd.DataFrame(columns=[f"score({self.Y.name})"])
        for key in self.trained_models.keys():
            accuracy.loc[key] = self.trained_models[key].score(X_test, y_test)
            
        self.accuracyScore = accuracy
        self.n_days = n_days
#         display(self.accuracyScore)
         
    def _scaleData(self):
        X_columns, X_values = self.X.select_dtypes(include=['float64']).columns,\
                              self.X.select_dtypes(include=['float64']).values
        X_scaled = min_max_scaler.fit_transform(X_values)
        self.X = self.X.drop(columns = X_columns)
        for i, column in enumerate(X_columns):
            self.X[column] = X_scaled.T[i]
    
    def _categorize(self, n_classes):
        max_value = self.Y.max()
        min_value = self.Y.min()
        binInterval = np.arange(min_value, max_value, (max_value-min_value)/n_classes).tolist()
        binInterval.append(max_value)
        binLabels   = range(0,len(binInterval)-1)
#         print(binLabels)
        self.Y = pd.cut(self.Y, bins = binInterval, labels=binLabels, include_lowest=True)
        print(set(self.Y))
        self.binIntervals = {i:[binInterval[i],binInterval[i+1]] for i in range(n_classes)}
        
    def predict(self, dayData):
        prediction = pd.DataFrame(columns=[f"prediction({self.Y.name})"])
        for i in range(1, self.n_days+1):
            pred = self.trained_models[i].predict([dayData])
            prediction.loc[i] = f"od {round(self.binIntervals[int(pred)][0],2)} do {round(self.binIntervals[int(pred)][1],2)}"\
                                if self.categorical else pred
        self.prediction = prediction
#         display(self.prediction)

In [95]:
dataset

,Średnie dobowe zachmurzenie ogólne [oktanty],Średnia dobowa prędkość wiatru [m/s],Średnia dobowe ciśnienie pary wodnej [hPa],Średnia dobowa wilgotność względna [%],Średnia dobowe ciśnienie na poziomie stacji [hPa],Suma opadu dzień [mm],Suma opadu noc [mm],Maksymalna temperatura dobowa [°C],Minimalna temperatura dobowa [°C],Suma dobowa opadu [mm],Temperatura minimalna przy gruncie [°C],Usłonecznienie [godziny],Średnia temperatura dobowa [°C],Czas trwania opadu deszczu [godziny],Czas trwania rosy [godziny],Czas trwania zamglenia [godziny],month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31
0,7.4,3.0,7.0,91.1,1007.1,0.60,0.0,4.6,2.8,0.6,2.3,0.6,3.2,6.75,0.0,13.2,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7.4,3.8,7.6,98.6,1010.8,0.10,0.1,5.1,2.1,0.2,0.3,1.3,3.2,5.70,0.0,7.9,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7.4,4.5,7.4,90.0,1008.3,0.75,0.5,7.3,2.2,3.5,1.7,0.0,4.2,6.75,0.0,6.9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,7.3,6.1,8.3,97.5,1011.8,0.40,0.5,8.4,1.6,1.2,-0.1,0.0,4.7,5.80,0.0,2.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4.3,3.3,7.0,85.5,1009.7,0.00,0.3,7.1,3.0,0.3,1.2,4.6,4.1,0.00,0.0,8.7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5093,6.8,3.9,5.5,78.3,1035.4,0.00,0.0,3.5,0.6,0.0,-0.1,1.3,1.8,0.70,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5094,6.5,8.0,4.4,65.9,998.5,0.00,0.5,3.1,-0.9,0.6,-0.9,0.0,1.4,2.20,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5095,6.1,5.1,4.7,60.0,1019.9,0.00,0.0,5.8,0.9,0.0,0.5,0.0,3.7,1.30,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5096,6.9,6.4,6.3,77.6,993.4,0.20,0.0,5.4,1.9,0.2,0.9,0.8,4.0,2.70,0.0,0.8,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [96]:
sun_clf_params = { 'n_jobs':-1,
              'max_depth' : params_best_sun['param_max_depth'].values,
              'min_samples_split' : int(params_best_sun['param_min_samples_split'].values),
              'n_estimators' : int(params_best_sun['param_n_estimators'].values),
              'class_weight' : "balanced"}
cloud_clf_params = {'n_estimators': int(params_best_clouds['n_estimators'].values),\
                    'learning_rate': params_best_clouds['learning_rate'].values[0]}
rain_clf_params = {'n_estimators': int(params_best_rain['n_estimators'].values)}
pressure_clf_params = {'n_estimators': int(params_best_pressure['n_estimators'].values),
                       'learning_rate': params_best_pressure['learning_rate'].values[0]}
humidity_clf_params = {'n_estimators': int(params_best_pressure['n_estimators'].values),
                       'learning_rate': params_best_pressure['learning_rate'].values[0]}


In [97]:
sun_fit_params = [3, 'RandomForestClassifier']
cloud_fit_params = [3, 'GradientBoostingClassifier']
rain_fit_params = [3, 'GradientBoostingClassifier']
pressure_fit_params = [3, 'GradientBoostingClassifier']
temp_fit_params = [None, None]
max_temp_fit_params = [None, None]
min_temp_fit_params = [None, None]
humidity_fit_params = [3, 'GradientBoostingClassifier']


In [98]:
columnsToPredict = {'Usłonecznienie [godziny]': sun_fit_params,
                    'Średnie dobowe zachmurzenie ogólne [oktanty]': cloud_fit_params,
                   'Suma dobowa opadu [mm]': rain_fit_params,
                    'Średnia dobowe ciśnienie na poziomie stacji [hPa]': pressure_fit_params,
                    'Średnia temperatura dobowa [°C]': temp_fit_params,
                    'Maksymalna temperatura dobowa [°C]': max_temp_fit_params,
                    'Minimalna temperatura dobowa [°C]': min_temp_fit_params,
                    'Średnia dobowe ciśnienie pary wodnej [hPa]': humidity_fit_params
                   }

In [99]:
columnsCLFParams = {'Usłonecznienie [godziny]': sun_clf_params,
                    'Średnie dobowe zachmurzenie ogólne [oktanty]': cloud_clf_params,
                   'Suma dobowa opadu [mm]': rain_clf_params,
                   'Średnia dobowe ciśnienie na poziomie stacji [hPa]': pressure_clf_params,
                    'Średnia temperatura dobowa [°C]':{},
                    'Maksymalna temperatura dobowa [°C]': {},
                    'Minimalna temperatura dobowa [°C]': {},
                    'Średnia dobowe ciśnienie pary wodnej [hPa]': humidity_clf_params
                   }

In [100]:
class WeatherPrediction:   
    def __init__(self, dataset, columnsToPredict, columnsCLFParams, n_days):
        self.dataset = dataset
        self.columnsToPredict = columnsToPredict
        self.n_days = n_days
        self.columnsCLFParams = columnsCLFParams
        self.models = {}
        self.scores = pd.DataFrame()
        self.prections = pd.DataFrame()
        
    def fit_data(self):
        for column in columnsToPredict.keys():
            model_clf = Classifier(self.dataset, dataset[column])
            model_clf.fit(self.n_days, *self.columnsToPredict[column], self.columnsCLFParams[column])
            self.scores[column] = model_clf.accuracyScore.values.flatten()
            self.models[column] = model_clf
        display(self.scores)
    
    def predict_days(self, dayData):
        for column in self.models.keys():
            self.models[column].predict(dayData)
            self.prections[column] = self.models[column].prediction.values.flatten()
        display(self.prections)

In [101]:
prediction = WeatherPrediction(dataset, columnsToPredict, columnsCLFParams, 7)

In [102]:
prediction.fit_data()

{'n_jobs': -1, 'max_depth': array([15], dtype=object), 'min_samples_split': 5, 'n_estimators': 300, 'class_weight': 'balanced'}
{0, 1, 2}
{'n_estimators': 16, 'learning_rate': 1.0}
{0, 1, 2}
{'n_estimators': 32}
{0, 1, 2}
{'n_estimators': 16, 'learning_rate': 0.8}
{0, 1, 2}
{'n_estimators': 16, 'learning_rate': 0.8}
{0, 1, 2}


,Usłonecznienie [godziny],Średnie dobowe zachmurzenie ogólne [oktanty],Suma dobowa opadu [mm],Średnia dobowe ciśnienie na poziomie stacji [hPa],Średnia temperatura dobowa [°C],Maksymalna temperatura dobowa [°C],Minimalna temperatura dobowa [°C],Średnia dobowe ciśnienie pary wodnej [hPa]
0,0.558523,0.516889,0.673213,0.648861,0.769775,0.765780,0.718944,0.756481
1,0.578947,0.530244,0.712490,0.369992,0.775990,0.764044,0.733999,0.758837
2,0.604085,0.547526,0.730558,0.654360,0.811490,0.815000,0.773893,0.778476
3,0.614297,0.554595,0.731343,0.394344,0.809771,0.798633,0.804466,0.798115
4,0.631579,0.570306,0.731343,0.682639,0.846587,0.849237,0.835202,0.801257
5,0.706206,0.597015,0.732914,0.399843,0.843396,0.841989,0.841962,0.809898
6,0.621367,0.552239,0.732914,0.633936,0.774220,0.784063,0.767547,0.776905


In [103]:
prediction.predict_days(data2020.iloc[0])

,Usłonecznienie [godziny],Średnie dobowe zachmurzenie ogólne [oktanty],Suma dobowa opadu [mm],Średnia dobowe ciśnienie na poziomie stacji [hPa],Średnia temperatura dobowa [°C],Maksymalna temperatura dobowa [°C],Minimalna temperatura dobowa [°C],Średnia dobowe ciśnienie pary wodnej [hPa]
0,od 5.4 do 10.8,od 0.0 do 2.67,od 0.0 do 1.17,od 961.2 do 988.13,5.614983,9.647016,2.866749,od 0.0 do 7.77
1,od 5.4 do 10.8,od 2.67 do 5.33,od 1.17 do 2.33,od 961.2 do 988.13,5.851375,8.994686,2.914330,od 0.0 do 7.77
2,od 0.0 do 5.4,od 2.67 do 5.33,od 2.33 do 3.5,od 961.2 do 988.13,4.688655,8.882386,2.725314,od 0.0 do 7.77
3,od 0.0 do 5.4,od 0.0 do 2.67,od 0.0 do 1.17,od 961.2 do 988.13,5.267793,9.214887,3.288757,od 0.0 do 7.77
4,od 0.0 do 5.4,od 2.67 do 5.33,od 0.0 do 1.17,od 988.13 do 1015.07,4.772154,10.664315,1.622312,od 0.0 do 7.77
5,od 0.0 do 5.4,od 2.67 do 5.33,od 1.17 do 2.33,od 961.2 do 988.13,6.995392,10.672274,1.862660,od 7.77 do 15.53
6,od 0.0 do 5.4,od 0.0 do 2.67,od 1.17 do 2.33,od 1015.07 do 1042.0,5.118334,8.635429,2.566763,od 0.0 do 7.77


In [104]:
data2020.iloc[0:7][prediction.prections.columns]

,Usłonecznienie [godziny],Średnie dobowe zachmurzenie ogólne [oktanty],Suma dobowa opadu [mm],Średnia dobowe ciśnienie na poziomie stacji [hPa],Średnia temperatura dobowa [°C],Maksymalna temperatura dobowa [°C],Minimalna temperatura dobowa [°C],Średnia dobowe ciśnienie pary wodnej [hPa]
0,7.7,5.5,0.0,994.9,6.9,10.9,3.1,7.6
1,1.6,5.3,0.3,995.4,4.3,8.6,0.3,7.6
2,0.0,7.1,4.6,997.2,4.9,6.5,0.3,8.5
3,2.5,5.8,0.0,1004.4,5.1,8.9,3.3,7.1
4,5.8,5.8,0.0,1005.5,3.3,7.8,-0.8,6.6
5,0.4,6.5,1.4,1000.1,4.5,7.4,1.5,6.4
6,0.0,8.0,2.2,1004.3,5.1,7.1,3.9,8.4
